### Peruse data

In [1]:
import pandas as pd
from datetime import datetime, date, timedelta

In [2]:
import re
def read_results_file(filename:str) -> pd.DataFrame:
    raw_df = pd.read_csv(filename) 
    # Seperate Day of Week into separate column
    raw_df['DayOfWeek'] = raw_df.Date.apply(lambda x : x.split()[0].replace("(","").replace(")",""))
    raw_df.Date = raw_df.Date.apply(lambda x : "-".join(x.split()[1:4]))
    raw_df.Date = pd.to_datetime(raw_df.Date, format="%d-%b-%Y")

    # Now rename columns and set index
    raw_df = raw_df[['Date', 'Team 1', 'FT', 'HT', 'Team 2', 'DayOfWeek']]
    raw_df = raw_df.rename(columns={'Date':'game_date','Team 1':'home_team','Team 2':'away_team'})
    raw_df = raw_df.set_index('game_date')

    # Break Full Time score into parts
    if 'FT' in raw_df.columns:
        df2 = pd.DataFrame(raw_df.FT.apply(lambda x: x.split('-')))
        raw_df[['score_home','score_away']] = pd.DataFrame(df2.FT.values.tolist(), index=df2.index)
        # Since we have parsed it, we can drop FT
        raw_df = raw_df.drop(columns=['FT'])

    # Break Full Time score into parts
    if 'HT' in raw_df.columns:
        df2 = pd.DataFrame(raw_df.HT.apply(lambda x: x.split('-')))
        raw_df[['half_score_home','half_score_away']] = pd.DataFrame(df2.HT.values.tolist(), index=df2.index)
        # Since we have parsed it, we can drop HT
        raw_df = raw_df.drop(columns=['HT'])

    # Simplify the strings
    raw_df['home_team'] = raw_df.home_team.apply(lambda x: x.split('(')[0].rstrip().lstrip())
    raw_df['away_team'] = raw_df.away_team.apply(lambda x: x.split('(')[0].rstrip().lstrip())
    return raw_df.reset_index().sort_values(by='game_date',ascending=True).set_index('game_date')

In [3]:
results_df = read_results_file("../../eng-england/2010s/2017-18/1-premierleague.csv")

In [6]:
#results_df

In [4]:
last_game_date=results_df.index[0] - timedelta(1)
for game in results_df.reset_index().values:
    if game[0] > last_game_date:
        last_game_date = game[0]
        # Before reading results of this new day, run a prediction_experiment based on the results
        # available till the day before
# At the end of the season there is no reason to run a prediction experiment but we might want to check
# how well we have done. However if the season is not complete, then we should run a final prediction